In [4]:
# Распред обручей
import pandas as pd
import numpy as np
import random

shops = pd.read_excel('C:\анаконда\shop_list.xlsx', dtype='int')
                     
# достаем Тпг+размер c количеством которые нужно считать из накладной 
tmp_df = pd.read_excel('C:/рушники/6321.xlsx', dtype='str')
tmp_df['tpg'] = tmp_df['Товарная подгруппа'].str.cat(tmp_df['Размер'], sep="_")

#del tmp_df['№ Поставщика']
tmp_df['tpg'] = tmp_df['tpg'].str.replace('.', ',')# меняем точки на запятые
tmp_df.sort_values(by=['tpg'], inplace=True)
tpg = tmp_df['tpg'].unique().tolist()
tpg.sort()
tpg_dic = tmp_df['tpg'].value_counts().to_dict()

# Загружаем нормы и остатки по нужным тпг чтобы посчитать дефицит
n_link =  r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx'
norms = pd.read_excel(n_link, sheet_name='гладкая_плоская размеры_тпг', skiprows=3, usecols=['номер', *tpg]) 
stock = pd.read_excel(r'C:\Остатки\Обручальные кольца.xlsx', usecols=['КодСклада','ТоварнаяПодгруппа','РазмерИзделия'], dtype='str')
stock.columns = ['КодСклада', 'Товарная подгруппа', 'Размер']
stock['Размер'] = stock['Размер'].str.replace('.', ',')
stock['tpg'] = stock['Товарная подгруппа'].str.cat(stock['Размер'], sep="_")
stock.drop(columns=['Товарная подгруппа','Размер'], inplace=True)
stock = stock[stock['tpg'].isin(tpg)]
stock = stock.groupby(by=['КодСклада', 'tpg'])['tpg'].aggregate('count').unstack(fill_value=0.0)
stock.reset_index(inplace=True)
stock = stock.astype('int')
stock.columns.values[0] = 'номер'
stock = stock[norms.columns.tolist()]
norms = shops.merge(norms, on='номер', how='left')
stock = shops.merge(stock, on='номер', how='left').fillna(0)
norms.set_index('номер', inplace=True)
stock.set_index('номер', inplace=True)

#norms = norms.applymap(lambda x: x + 1 if x == 0 else round(x * 1.3))# для распреда того что не входит

demand = norms - stock
demand = demand.applymap(lambda num: 0 if num < 0 else num)

def raspred_dict(df):
    emp = []
    cols = df.columns.tolist()
    for col in cols:
        emp.append(list(np.repeat(df.index.to_numpy(),df[col].values.astype(int))))
    return dict(zip(cols, emp))

no7171_dict = raspred_dict(demand)

def dictu(x):    
    if tpg_dic[x] > len(no7171_dict[x]):
        over = tpg_dic[x] - len(no7171_dict[x])
        no7171_dict[x] = [*no7171_dict[x], *([7171] * over)]        
    elif tpg_dic[x] < len(no7171_dict[x]):
        no7171_dict[x] = no7171_dict[x][:tpg_dic[x]]        
    return no7171_dict[x]

lst = [dictu(x) for x in tpg]

tmp_df['Период реализации'] = np.array([item for sublist in lst for item in sublist])
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
tmp_df.to_excel(name, sheet_name='Движение товара', index=False)

In [3]:
# join files in folder
import os
import pandas as pd

path = 'C:/рушники/соединить'
  
files = [path + '/' + x for x in os.listdir(path)]
all_files = pd.concat([pd.read_excel(x) for x in files])
all_files.to_excel(path + '/all.xlsx', sheet_name='Движение товара', index=False)

In [8]:
# Распред букв или зодиаков по одному направлению
import pandas as pd
import numpy as np
import random

# get stockdata, goods receipt, shops list, stock capability
prih = pd.read_excel('C:/рушники/ИФ ЗОДИАК.xlsx')
prih.sort_values(by=['Товар'], inplace=True)
prih['Описание 3'] = prih['Описание 3'].str.lower()

letters = prih['Описание 3'].unique().tolist()
tovs = prih['Товар'].unique().tolist()
tovs.sort()

stock = pd.read_excel('C:/Остатки/Буквы и Зодиаки.xlsx')
stock = stock[stock['ТоварНо'].isin(tovs)]
stock.rename(columns={'Описание3': 'Описание 3'}, inplace=True)
stock['Описание 3'] = stock['Описание 3'].str.lower()

#create automatic demand calculation here
#norms = pd.read_excel('C:/Остатки/norm_bz.xlsx', sheet_name='norm_z', index_col= 'Название из NAV', usecols=['Название из NAV', 'sum_bk'])
#norm_dict = norms.sum_bk.apply(lambda x: round(x/12)).to_dict()

#estimate sheetname and filename for demand
tn = prih.loc[1, 'Товарное направление'].lower()
fname = 'C:/Остатки/' + prih.loc[1, 'Дизайн'][3:].lower() + '.xlsx'

demand = pd.read_excel(fname, sheet_name=tn, skiprows=1, usecols=['Наполняха', 'Код', *letters], index_col='Код')


#Create two dictionaries tov dict and letter dict
stock_dict = dict(zip(tovs,[stock[stock['ТоварНо'] == tov]['КодСклада'].tolist() for tov in tovs]))
df = prih.groupby(by=[ 'Товар', 'Описание 3'])['Товар'].count().rename('count')
df = df.reset_index()

tov_dict = dict(zip(df.Товар.tolist(), df[['Описание 3', 'count']].values.tolist())) #(tov:[letter, demand])

#letter: dataframe with columns:store number, filling rate, demand(positive num is overstock, negative num is demand)
letter_dict = dict(zip(letters,[demand[['Наполняха', letter]] for letter in letters]))

def get_shoplist(tov):
    """Get sorted by letter demand and stock list of stores with zero stock of tov.
    Changes letter dict according to this list. Two dictionaries needed to perform func"""
    
    letter = tov_dict[tov][0]
    lenth = tov_dict[tov][1]
    shoplist_full = letter_dict[letter].sort_values(by=[letter, 'Наполняха']).index.tolist()
    shoplist = [i for i in shoplist_full if i not in stock_dict[tov]]
    shop_len = len(shoplist)
    
    if shop_len > lenth:
        shoplist = shoplist[:lenth]
    elif len(shoplist) < lenth:
        over = lenth - shop_len
        shoplist = [*shoplist,*([7171] * over)]
    for shop in shoplist:
        if shop != 7171:
            letter_dict[letter][letter][shop] += 1
    return shoplist
      
lst = [get_shoplist(x) for x in tovs]
prih['Период реализации'] = np.array([item for sublist in lst for item in sublist])

name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
prih.to_excel(name, sheet_name='Движение товара', index=False)
    

C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
# Распред по товам по две три штуки в шоплист вторым столбцом указать количество столбец "кол"
import pandas as pd
import numpy as np
import random

# create distribution quantities dictionary
invoice = pd.read_excel('C:/рушники/123.xlsx')
invoice.sort_values(by=['Товар'], inplace=True)
arts_dic = invoice['Товар'].value_counts().to_dict()

# create norms dataframe
shops = pd.read_excel('C:\анаконда\shop_list_doubled.xlsx', dtype='int')
shops.sort_values(by='кол',ascending=False, inplace=True)

num_dict = shops.кол.value_counts().sort_index(ascending=False).to_dict()
cols = list(invoice['Товар'].unique())

norms = pd.concat(
[pd.DataFrame(
    data=np.full((num_dict[i], len(cols)), i), 
    columns=cols)
 for i in list(num_dict)]
)  

norms.index = shops['номер']

# create stock df identical in shape to norms df
stock = pd.read_excel(r'C:\Остатки\Товы.xlsx', usecols=['КодСклада','ТоварНо'])
stock = stock[stock['КодСклада'].isin(norms.index.tolist()) & 
              stock['ТоварНо'].isin(cols)]

pivot = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()

stock = pd.DataFrame(data=shops['номер'].values, columns=['КодСклада']).merge(pivot, how='left')
stock.set_index('КодСклада',inplace=True)
stock.fillna(0, inplace=True)

if cols != stock.columns.to_list():
    add_cols = [x for x in cols if x not in stock.columns.to_list()]
    add_df = pd.DataFrame(data=np.zeros((stock.shape[0], len(add_cols))), columns=add_cols, index=stock.index)
    stock = stock.join(add_df)
    
stock = stock[cols]

# demand 
demand = norms - stock

# distribution
def repeat_series(series):
    return np.repeat(series.index, series.values).tolist()

empty = []
for col in demand.columns.tolist():
    shop_lst = repeat_series(demand[col].loc[lambda x: x > 0])
    diff = arts_dic[col] - len(shop_lst)
    if diff > 0:
        shop_lst.extend(list(np.full(diff, 7171)))
    else:
        shop_lst = shop_lst[:arts_dic[col]]
    empty.extend(shop_lst)

invoice['Период реализации'] = empty

# save and exit
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
invoice.to_excel(name, sheet_name='Движение товара', index=False)

In [142]:
#Делим вырицу
path = 'C:/рушники/вырица.xlsx'
df = pd.read_excel(path, index_col=0)
df.fillna(0, inplace=True, downcast='infer') #либо менять dtype
df.sort_index(inplace=True)
cols = df.columns.tolist()

empty = []
for x in df.index:
    lst = np.repeat(cols, df.loc[x].values)
    empty = np.append(empty, lst)
    
path = 'C:/рушники/123.xlsx'
df1 = pd.read_excel(path)
df1.sort_values(by='Артикул', inplace=True)
df1['Период реализации'] = empty

# save and exit
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
df1.to_excel(name, sheet_name='Движение товара', index=False)

In [4]:
# РАСПРЕД ПО МАГАМ ТОПАМ
import pandas as pd
import numpy as np

n_of_shops = 50

invoice = 'C:/рушники/123.xlsx'
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

# накладная сортировка, сводная ТГ-тов-кол-во для распреда
prih = pd.read_excel(invoice)
prih['Тип изделия 1'] = prih['Тип изделия 1'].apply(lambda x: 'СЕРЬГИ' if x in ['ПИРСИНГ', 'СЕРЬГИ-КОНГО'] else x)
prih['Описание 3'] = prih['Товарное направление'] + '_' + prih['Тип изделия 1']
prih.sort_values(by=['Описание 3', 'Товар'], inplace=True)

prih_summary = pd.DataFrame(prih.groupby(by=['Описание 3', 'Товар'])['Товар'].count().rename('quantity')).reset_index()
prih_summary.sort_values(by=['Описание 3', 'Товар'], inplace=True)

# {ТГ: [магазины с большими нормами по тг, разрешенные к распреду, наполняемость<100%, сортировка по наполняемости]}
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

lst = pd.read_excel(fpath, skiprows=2,nrows=0).columns.tolist()
lst = [str(x) for x in lst]#защита от ошибки если в ячейке цифра
point_column = max(i for i, v in enumerate(lst) if 'Кольцо' in v)

cols = [1, 2] + list(range(4, 10)) + list(range(point_column, point_column + 6))
#создаем список тг
type1 = ['КОЛЬЦО',
'КОЛЬЦО ОБРУЧАЛЬНОЕ',
'КОЛЬЦО ПЕЧАТКА',
'ПОДВЕС ДЕКОРАТИВНЫЙ',
'ПОДВЕС КУЛЬТОВЫЙ',
'СЕРЬГИ']

tg_list = []

for tn in ['ИФ', 'БК']:
    for t in type1:
        tg_list.append(tn + '_' + t)

def read_norms(tn):
    """Создает датафрейм по ТН с наполняемостью и нормами по ТГ, франчайзинг, закрытые и мусульманские удаляются"""
    names = [i for i in tg_list if tn in i]
    df = pd.read_excel(fpath, sheet_name=tn, skiprows=3, usecols=cols, index_col='Код')
    df = df[~df.блок.isin(['ф', 'м', 'з', 'д', 'м/з'])].drop(columns='блок')
    arrays = [['Нормы','Наполняемость'], names]
    df.columns = pd.MultiIndex.from_product(arrays)
    return df

IF, BK = read_norms('ИФ'), read_norms('БК')

"""Два варианта отбора магов: 1 - сначала выбрать количество по нормам, а потом убрать с наполняемостью меньше 100%; 
2 - убрать маги меньше 100% и выбрать количество. В 1 случае список магов к каждому ТГ будет разной длины и нужно делать 
словарь; во 2 случае списки к каждому тг одинаковой длины и можно сделать датафрейм. Реализуется 2 случай -  
датафрейм из словаря"""

df = IF.join(BK, how='outer')
shop_lsts = {}
keys = prih_summary['Описание 3'].unique()
for x in keys:
    filt = df.xs(x, level=1, axis=1).sort_values(by='Нормы', ascending=False).head(50)
    shop_lsts[x] = filt.index[filt['Наполняемость'] < 1.1].to_list()#наполняху менять тут

# Остатки реальные + накладные
cols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"]
stock = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False, usecols=cols)
stock = stock[stock.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del stock['Проект']

stock.dropna(inplace=True)
stock.КодСклада = stock.КодСклада.astype('int64')

stock_centr = stock[stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[~stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[["КодСклада", "ТоварНо", "СерийныйНомер"]]
del stock_centr['КодСклада']

cols=["Серийный Но.","Склад-приемник","Номер ЗЛ"]
zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4, usecols=cols).sort_values(by="Номер ЗЛ",ascending = False)
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

stock_centr = pd.merge(stock_centr,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
stock_centr = stock_centr[["КодСклада",  "ТоварНо","СерийныйНомер"]]
stock = pd.concat([stock,stock_centr])
#оставим только маги-топы и товы из накладной
unique_lst = list(set([val for vals in shop_lsts.values() for val in vals]))
stock = stock[stock.КодСклада.isin(unique_lst)].drop(columns='СерийныйНомер')
stock = stock[stock.ТоварНо.isin(prih_summary.Товар.tolist())]
#формат колонок кодсклада и товы
stock = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()
stock = pd.DataFrame({'КодСклада': unique_lst}).merge(stock, on='КодСклада',how='left')#добавил
stock = stock.rename_axis(None, axis=1).set_index('КодСклада')
if set(prih_summary.Товар.values) != set(stock.columns):
    cols = set(prih_summary.Товар.values) - set(stock.columns)
    stock = stock.assign(**dict(zip(list(cols), np.zeros(len(cols), dtype='int64'))))

# Распределение
def distribution(tg):
    """распред по списку магов ТГ"""
    tov_dict = dict(prih_summary[prih_summary['Описание 3'] == tg].iloc[:,[1, 2]].values)
    cols = list(tov_dict.keys())    
    shops = shop_lsts[tg]
    tn_stock = stock.loc[shops, cols]
    emp_lst = []
    for col in tn_stock.columns.tolist():
        shop_lst = tn_stock[col].loc[lambda x: x == 0].index.tolist()
        lenth = len(shop_lst)
        diff = tov_dict[col] - lenth
        
        if diff > 0:
            shop_lst.extend(list(np.full(diff, 7171)))
        else:
            shop_lst = shop_lst[:tov_dict[col]]
            
        emp_lst.extend(shop_lst)
    return emp_lst
    
distr_lst = []
for x in list(shop_lsts.keys()):
    distr_lst.extend(distribution(x))

prih['Период реализации'] = distr_lst

# save and exit
name = 'C:/рушники/' + str(np.random.randint(5000, 10000)) + '.xlsx'
prih.to_excel(name, sheet_name='Движение товара', index=False)

In [5]:
# РАСПРЕД ПО всем магам
import pandas as pd
import numpy as np

invoice = 'C:/рушники/7982.xlsx'
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

# накладная сортировка, сводная ТГ-тов-кол-во для распреда
prih = pd.read_excel(invoice)
prih['Тип изделия 1'] = prih['Тип изделия 1'].apply(lambda x: 'СЕРЬГИ' if x in ['ПИРСИНГ', 'СЕРЬГИ-КОНГО'] else x)
prih['Описание 3'] = prih['Товарное направление'] + '_' + prih['Тип изделия 1']
prih.sort_values(by=['Описание 3', 'Товар'], inplace=True)

prih_summary = pd.DataFrame(prih.groupby(by=['Описание 3', 'Товар'])['Товар'].count().rename('quantity')).reset_index()
prih_summary.sort_values(by=['Описание 3', 'Товар'], inplace=True)

# {ТГ: [магазины, разрешенные к распреду, сортировка по наполняемости]}
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

lst = pd.read_excel(fpath, skiprows=2,nrows=0).columns.tolist()
lst = [str(x) for x in lst]#защита от ошибки если в ячейке цифра
point_column = max(i for i, v in enumerate(lst) if 'Кольцо' in v)

cols = [1, 2] + list(range(point_column, point_column + 12))
#создаем список тг
type1 = ['КОЛЬЦО',
'КОЛЬЦО ОБРУЧАЛЬНОЕ',
'КОЛЬЦО ПЕЧАТКА',
'ПОДВЕС ДЕКОРАТИВНЫЙ',
'ПОДВЕС КУЛЬТОВЫЙ',
'СЕРЬГИ']

tg_list = []

for tn in ['ИФ', 'БК']:
    for t in type1:
        tg_list.append(tn + '_' + t)

def read_norms(tn):
    """Создает датафрейм по ТН с наполняемостью и разрешениями по ТГ, франчайзинг, закрытые и мусульманские удаляются"""
    names = [i for i in tg_list if tn in i]
    df = pd.read_excel(fpath, sheet_name=tn, skiprows=3, usecols=cols, index_col='Код')
    df = df[~df.блок.isin(['ф', 'м', 'з', 'д', 'м/з'])].drop(columns='блок')
    arrays = [['Нормы','Наполняемость'], names]
    df.columns = pd.MultiIndex.from_product(arrays)
    return df

IF, BK = read_norms('ИФ'), read_norms('БК')

"""Два варианта отбора магов: 1 - сначала выбрать количество по нормам, а потом убрать с наполняемостью меньше 100%; 
2 - убрать маги меньше 100% и выбрать количество. В 1 случае список магов к каждому ТГ будет разной длины и нужно делать 
словарь; во 2 случае списки к каждому тг одинаковой длины и можно сделать датафрейм. Реализуется 1 случай -  
датафрейм из словаря"""

df = IF.join(BK, how='outer')
shop_lsts = {}
keys = prih_summary['Описание 3'].unique()
for x in keys:
    idx = pd.IndexSlice
    filtr = df.loc[:, idx['Наполняемость',x]] == 'нет'
    shop_lsts[x] = df.xs(x, level=1, axis=1)[filtr].sort_values(by='Нормы').index.to_list()

# Остатки реальные + накладные
cols = ["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"]
stock = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False, usecols=cols)
stock = stock[stock.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del stock['Проект']

stock.dropna(inplace=True)
stock.КодСклада = stock.КодСклада.astype('int64')

stock_centr = stock[stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[~stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[["КодСклада", "ТоварНо", "СерийныйНомер"]]
del stock_centr['КодСклада']

cols=["Серийный Но.","Склад-приемник","Номер ЗЛ"]
zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4, usecols=cols).sort_values(by="Номер ЗЛ",ascending = False)
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

stock_centr = pd.merge(stock_centr,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
stock_centr = stock_centr[["КодСклада",  "ТоварНо","СерийныйНомер"]]
stock = pd.concat([stock,stock_centr])
#оставим только маги-топы и товы из накладной
unique_lst = list(set([val for vals in shop_lsts.values() for val in vals]))
stock = stock[stock.КодСклада.isin(unique_lst)].drop(columns='СерийныйНомер')
stock = stock[stock.ТоварНо.isin(prih_summary.Товар.tolist())]
#формат колонок кодсклада и товы
stock = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()
stock = pd.DataFrame({'КодСклада': unique_lst}).merge(stock, on='КодСклада',how='left')#добавил
stock = stock.rename_axis(None, axis=1).set_index('КодСклада')
if set(prih_summary.Товар.values) != set(stock.columns):
    cols = set(prih_summary.Товар.values) - set(stock.columns)
    stock = stock.assign(**dict(zip(list(cols), np.zeros(len(cols), dtype='int64'))))

# Распределение
def distribution(tg):
    """распред по списку магов ТГ"""
    tov_dict = dict(prih_summary[prih_summary['Описание 3'] == tg].iloc[:,[1, 2]].values)
    cols = list(tov_dict.keys())    
    shops = shop_lsts[tg]
    tn_stock = stock.loc[shops, cols]
    emp_lst = []
    for col in tn_stock.columns.tolist():
        shop_lst = tn_stock[col].loc[lambda x: x == 0].index.tolist()
        lenth = len(shop_lst)
        diff = tov_dict[col] - lenth
        
        if diff > 0:
            shop_lst.extend(list(np.full(diff, 7171)))
        else:
            shop_lst = shop_lst[:tov_dict[col]]
            
        emp_lst.extend(shop_lst)
    return emp_lst
    
distr_lst = []
for x in list(shop_lsts.keys()):
    distr_lst.extend(distribution(x))

prih['Период реализации'] = distr_lst

# save and exit
name = 'C:/рушники/' + str(np.random.randint(5000, 10000)) + '.xlsx'
prih.to_excel(name, sheet_name='Движение товара', index=False)

In [73]:
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

lst = pd.read_excel(fpath, skiprows=2,nrows=0).columns.tolist()
lst = [str(x) for x in lst]#защита от ошибки если в ячейке цифра
point_column = 206

cols = [1, 2] + list(range(4, 10)) + list(range(point_column, point_column + 6))
#создаем список тг
type1 = ['КОЛЬЦО',
'КОЛЬЦО ОБРУЧАЛЬНОЕ',
'КОЛЬЦО ПЕЧАТКА',
'ПОДВЕС ДЕКОРАТИВНЫЙ',
'ПОДВЕС КУЛЬТОВЫЙ',
'СЕРЬГИ']

tg_list = []

for tn in ['ИФ', 'БК']:
    for t in type1:
        tg_list.append(tn + '_' + t)

def read_norms(tn):
    """Создает датафрейм по ТН с наполняемостью и нормами по ТГ, франчайзинг, закрытые и мусульманские удаляются"""
    names = [i for i in tg_list if tn in i]
    df = pd.read_excel(fpath, sheet_name=tn, skiprows=3, usecols=cols, index_col='Код')
    df = df[~df.блок.isin(['ф', 'м', 'з', 'д', 'м/з'])].drop(columns='блок')
    arrays = [['Нормы','Наполняемость'], names]
    df.columns = pd.MultiIndex.from_product(arrays)
    return df

IF, BK = read_norms('ИФ'), read_norms('БК')

In [76]:
df = IF.join(BK, how='outer')
df.columns

MultiIndex([(        'Нормы',              'ИФ_КОЛЬЦО'),
            (        'Нормы',  'ИФ_КОЛЬЦО ОБРУЧАЛЬНОЕ'),
            (        'Нормы',      'ИФ_КОЛЬЦО ПЕЧАТКА'),
            (        'Нормы', 'ИФ_ПОДВЕС ДЕКОРАТИВНЫЙ'),
            (        'Нормы',    'ИФ_ПОДВЕС КУЛЬТОВЫЙ'),
            (        'Нормы',              'ИФ_СЕРЬГИ'),
            ('Наполняемость',              'ИФ_КОЛЬЦО'),
            ('Наполняемость',  'ИФ_КОЛЬЦО ОБРУЧАЛЬНОЕ'),
            ('Наполняемость',      'ИФ_КОЛЬЦО ПЕЧАТКА'),
            ('Наполняемость', 'ИФ_ПОДВЕС ДЕКОРАТИВНЫЙ'),
            ('Наполняемость',    'ИФ_ПОДВЕС КУЛЬТОВЫЙ'),
            ('Наполняемость',              'ИФ_СЕРЬГИ'),
            (        'Нормы',              'БК_КОЛЬЦО'),
            (        'Нормы',  'БК_КОЛЬЦО ОБРУЧАЛЬНОЕ'),
            (        'Нормы',      'БК_КОЛЬЦО ПЕЧАТКА'),
            (        'Нормы', 'БК_ПОДВЕС ДЕКОРАТИВНЫЙ'),
            (        'Нормы',    'БК_ПОДВЕС КУЛЬТОВЫЙ'),
            (        'Нормы',  

In [78]:
tg_list

['ИФ_КОЛЬЦО',
 'ИФ_КОЛЬЦО ОБРУЧАЛЬНОЕ',
 'ИФ_КОЛЬЦО ПЕЧАТКА',
 'ИФ_ПОДВЕС ДЕКОРАТИВНЫЙ',
 'ИФ_ПОДВЕС КУЛЬТОВЫЙ',
 'ИФ_СЕРЬГИ',
 'БК_КОЛЬЦО',
 'БК_КОЛЬЦО ОБРУЧАЛЬНОЕ',
 'БК_КОЛЬЦО ПЕЧАТКА',
 'БК_ПОДВЕС ДЕКОРАТИВНЫЙ',
 'БК_ПОДВЕС КУЛЬТОВЫЙ',
 'БК_СЕРЬГИ']

In [80]:
df = IF.join(BK, how='outer')
emp = []
for x in tg_list:
    emp.append(df.xs(x, level=1, axis=1).sort_values(by='Нормы', ascending=False).head(50)[['Наполняемость']].reset_index())
    

In [84]:
new_df = pd.concat(emp, axis=1)


In [101]:
new_df.columns

Index(['Код', 'Наполняемость', 'Код', 'Наполняемость', 'Код', 'Наполняемость',
       'Код', 'Наполняемость', 'Код', 'Наполняемость', 'Код', 'Наполняемость',
       'Код', 'Наполняемость', 'Код', 'Наполняемость', 'Код', 'Наполняемость',
       'Код', 'Наполняемость', 'Код', 'Наполняемость', 'Код', 'Наполняемость'],
      dtype='object')

In [105]:
tg_list*2

['ИФ_КОЛЬЦО',
 'ИФ_КОЛЬЦО ОБРУЧАЛЬНОЕ',
 'ИФ_КОЛЬЦО ПЕЧАТКА',
 'ИФ_ПОДВЕС ДЕКОРАТИВНЫЙ',
 'ИФ_ПОДВЕС КУЛЬТОВЫЙ',
 'ИФ_СЕРЬГИ',
 'БК_КОЛЬЦО',
 'БК_КОЛЬЦО ОБРУЧАЛЬНОЕ',
 'БК_КОЛЬЦО ПЕЧАТКА',
 'БК_ПОДВЕС ДЕКОРАТИВНЫЙ',
 'БК_ПОДВЕС КУЛЬТОВЫЙ',
 'БК_СЕРЬГИ',
 'ИФ_КОЛЬЦО',
 'ИФ_КОЛЬЦО ОБРУЧАЛЬНОЕ',
 'ИФ_КОЛЬЦО ПЕЧАТКА',
 'ИФ_ПОДВЕС ДЕКОРАТИВНЫЙ',
 'ИФ_ПОДВЕС КУЛЬТОВЫЙ',
 'ИФ_СЕРЬГИ',
 'БК_КОЛЬЦО',
 'БК_КОЛЬЦО ОБРУЧАЛЬНОЕ',
 'БК_КОЛЬЦО ПЕЧАТКА',
 'БК_ПОДВЕС ДЕКОРАТИВНЫЙ',
 'БК_ПОДВЕС КУЛЬТОВЫЙ',
 'БК_СЕРЬГИ']

In [109]:
pd.MultiIndex.from_arrays([ tg_list,new_df.columns])

ValueError: all arrays must be same length

In [110]:
new_df.columns = pd.MultiIndex.from_arrays([[x for pair in zip(tg_list, tg_list) for x in pair],new_df.columns])

In [113]:

name = 'C:/рушники/' + str(np.random.randint(5000, 10000)) + '.xlsx'
new_df.to_excel(name)

In [128]:
df = pd.read_excel(r'C:\Users\Dotsenko.Semen\Downloads\тестовые задания_с моделью.xlsx', sheet_name='info', header=[0,1], index_col=0)

In [164]:
df = df.astype('int')

In [165]:
df

Июн                                          \
          Кол-во плательщиков Количество      Сумма Сумма компенсации   
Продукт 1                3048       3053     406993                 0   
Продукт 2               28412     136693  392593368                 0   
Продукт 3                4637       5125    1025780                 0   
Продукт 4                 919       2112     392252                 0   
Продукт 5                 687        851     672010                 0   
Продукт 6                 833       2415     673019                 0   
Продукт 7                 106        114     344826                 0   
Все без 2               10230      13670    3514880                 0   

                                             Авг                        \
          Чистый доход_итого Кол-во плательщиков Количество      Сумма   
Продукт 1               5314                2060       2060     489873   
Продукт 2            4381845               26180     128903  370740595   
Продукт 3               9358                4163       4678    1137100   
Продукт 4               4925                 899       2080     400312   
Продукт 5               8685                 841       1017    1030267   
Продукт 6               8144                 833       1929     693420   
Продукт 7               5209                 321        373    1265700   
Все без 2              41638                9117      12137    5016672   

                                                               Сен             \
          Сумма компенсации Чистый доход_итого Кол-во плательщиков Количество   
Продукт 1             48986               6440                2383       2383   
Продукт 2                 0            4166107               26147     129320   
Продукт 3            113710              12482                4521       5033   
Продукт 4             40021               5100                 949       2394   
Продукт 5            102780              13544                1917       2361   
Продукт 6             65969               8481                 920       2458   
Продукт 7            105807              17507                 350        398   
Все без 2            477275              63557               11040      15027   

                                                                          Окт  \
               Сумма Сумма компенсации Чистый доход_итого Кол-во плательщиков   
Продукт 1     514386             51419               6688                2505   
Продукт 2  373341611                 0            4154876               49269   
Продукт 3    1385020            138502              15406                4893   
Продукт 4     412148             41178               5015                1440   
Продукт 5    2250656            225065              29263                2382   
Продукт 6     788643             71802               9723                1344   
Продукт 7    1328111            113166              18653                 336   
Все без 2    6678964            641134              84750               12900   

                                                                      
          Количество      Сумма Сумма компенсации Чистый доход_итого  
Продукт 1       2505     645828             64582               8397  
Продукт 2     139035  402662380                 0            4449213  
Продукт 3       5310    1701660            170166              19237  
Продукт 4       2892     393228             39293               4898  
Продукт 5       2556    2609400            256426              33927  
Продукт 6       2862     940062             71991              11232  
Продукт 7        363    2168488            186408              28366  
Все без 2      16488    8458666            788867             106058

In [176]:
idx = pd.IndexSlice

sum_pers = df.loc[:, idx[:, 'Сумма']].values / df.loc[:, idx[:, 'Кол-во плательщиков']].values
trans_pers = df.loc[:, idx[:, 'Количество']].values / df.loc[:, idx[:, 'Кол-во плательщиков']].values
indx = df.index
col_sum = pd.MultiIndex.from_tuples([(x[0], 'Сумма на человека')  for x in df.loc[:, idx[:, 'Сумма']].columns.tolist()])
col_trans = pd.MultiIndex.from_tuples([(x[0], 'Транзакц на человека')  for x in df.loc[:, idx[:, 'Сумма']].columns.tolist()])



In [177]:
to_join = pd.DataFrame(data=sum_pers, index=indx, columns=col_sum).join(pd.DataFrame(data=trans_pers, index=indx, columns=col_trans))

In [178]:
to_join

,Июн,Авг,Сен,Окт,Июн,Авг,Сен,Окт
,Сумма на человека,Сумма на человека,Сумма на человека,Сумма на человека,Транзакц на человека,Транзакц на человека,Транзакц на человека,Транзакц на человека
Продукт 1,133.527887,237.802427,215.856483,257.815569,1.001640,1.000000,1.000000,1.000000
Продукт 2,13817.871604,14161.214477,14278.563927,8172.732956,4.811101,4.923720,4.945883,2.821957
Продукт 3,221.216304,273.144367,306.352577,347.774372,1.105240,1.123709,1.113249,1.085224
Продукт 4,426.824810,445.285873,434.297155,273.075000,2.298150,2.313682,2.522655,2.008333
Продукт 5,978.180495,1225.049941,1174.051122,1095.465995,1.238719,1.209275,1.231612,1.073048
Продукт 6,807.945978,832.436975,857.220652,699.450893,2.899160,2.315726,2.671739,2.129464
Продукт 7,3253.075472,3942.990654,3794.602857,6453.833333,1.075472,1.161994,1.137143,1.080357
Все без 2,343.585533,550.254689,604.978623,655.710543,1.336266,1.331249,1.361141,1.278140


In [211]:
quant = df.loc[('Продукт 2', 'Все без 2'), idx[:, ('Кол-во плательщиков', 'Количество')]]


In [212]:
quant.columns = quant.columns.swaplevel()

In [213]:
quant

,Кол-во плательщиков,Количество,Кол-во плательщиков,Количество,Кол-во плательщиков,Количество,Кол-во плательщиков,Количество
,Июн,Июн,Авг,Авг,Сен,Сен,Окт,Окт
Продукт 2,28412,136693,26180,128903,26147,129320,49269,139035
Все без 2,10230,13670,9117,12137,11040,15027,12900,16488


In [216]:
quant.sort_index(axis=1, level=0, sort_remaining=False)

Кол-во плательщиков                      Количество                  \
                          Июн    Авг    Сен    Окт        Июн     Авг     Сен   
Продукт 2               28412  26180  26147  49269     136693  128903  129320   
Все без 2               10230   9117  11040  12900      13670   12137   15027   

                   
              Окт  
Продукт 2  139035  
Все без 2   16488

In [236]:
money = df.loc[('Продукт 2', 'Все без 2'), idx[:, ('Сумма', 'Чистый доход_итого')]]
money.columns = money.columns.swaplevel()
money = money.sort_index(axis=1, level=0, sort_remaining=False)

add = to_join.loc[('Продукт 2', 'Все без 2'), idx[:, ('Сумма на человека')]]
add.columns = add.columns.swaplevel()



In [237]:
money.join(add)

Сумма                                  Чистый доход_итого  \
                 Июн        Авг        Сен        Окт                Июн   
Продукт 2  392593368  370740595  373341611  402662380            4381845   
Все без 2    3514880    5016672    6678964    8458666              41638   

                                     Сумма на человека                \
               Авг      Сен      Окт               Июн           Авг   
Продукт 2  4166107  4154876  4449213      13817.871604  14161.214477   
Все без 2    63557    84750   106058        343.585533    550.254689   

                                      
                    Сен          Окт  
Продукт 2  14278.563927  8172.732956  
Все без 2    604.978623   655.710543

In [229]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
sns.relplot(money)

ValueError: Either both or neither of `x` and `y` must be specified (but try passing to `data`, which is more flexible).